#### Setup

A `changepoint` is an underlying shift in the parameters that generate a data sequence (e.g. the mean of a Gaussian suddenly jumps). Here, we focus on the online or causal task

To do this, we introduce a latent variable at time step $t$ named `run length` $r_t$, indicating the number of steps since the most recent changepoint

Based on observation $x_t$, if a changepoint occurs at $t$, then $r_t=0$; otherwise it increments by 1 from $r_{t-1}$

We want to maintains full run length `posterior` $p(r_t|x_{1:t})$ and update this `recursively`

In addition, we want to update the sequence `predictive distribution` for $x_{t+1}$ using only $x_{1:t}$


#### Predictive distribution

Using marginalization, we can write the `predictive` as

$$
\begin{align*}
p(x_{t+1}|x_{1:t})&=\sum_{r_t=0}^tp(x_{t+1}|r_t, x_{t-r_t:t})p(r_t|x_{1:t}) \\
&=\sum_{r_t=0}^tp(x_{t+1}|r_t, x_t^{(r)})p(r_t|x_{1:t})
\end{align*}
$$

where $x_t^{(r)}$ denotes the portion of $x_{1:t}$ that belongs to the `current` run

This shows that sequence predictive is determined by $p(x_{t+1}|r_t, x_t^{(r)})$ and run length posterior $p(r_t|x_{1:t})$

$p(x_{t+1}|r_t, x_t^{(r)})$ is often refered to as underlying probabilistic model `(UPM) predictive` to distinguish it from sequence predictive $p(x_{t+1}|x_{1:t})$

Next, we look at run length posterior and UPM predictive

#### Run length posterior

To compute run length posterior, we use the expression of conditional probability

$$\begin{align*}
p(r_t|x_{1:t})&=\frac{p(r_t, x_{1:t})}{\sum_{r_{t'}}p(r_{t'},x_{1:t})}
\end{align*}$$

We express the joint in a `recursive` manner

$$\begin{align*}
p(r_t, x_{1:t})&=\sum_{r_{t-1}}p(r_t, r_{t-1}, x_t, x_{1:t-1}) \\
&=\sum_{r_{t-1}}p(r_t, x_t | r_{t-1},  x_{1:t-1})p(r_{t-1}, x_{1:t-1})\\
&=\sum_{r_{t-1}}p(x_t|r_t, r_{t-1}, x_{1:t-1})p(r_t|r_{t-1}, x_{1:t-1})p(r_{t-1}, x_{1:t-1}) \\
& \text{assumption: } x_t \text{ conditionally independent of } r_t \\
& \text{assumption: } r_t \text{ conditionally independent of } x_{1:t-1} \\
&=\sum_{r_{t-1}}p(x_t|r_{t-1}, x_{1:t-1})p(r_t|r_{t-1})p(r_{t-1}, x_{1:t-1})\\
&=\sum_{r_{t-1}}p(x_t|r_{t-1}, x_{t-1}^{(r)})p(r_t|r_{t-1})p(r_{t-1}, x_{1:t-1})\\
\end{align*}$$

Therefore, the joint at step $t$, $p(r_t, x_{1:t})$, depends on UPM predictive $p(x_t|r_{t-1}, x_{t-1}^{(r)})$ and joint $p(r_{t-1}, x_{1:t-1})$ at step $t-1$ plus a changepoint prior $p(r_t|r_{t-1})$

It is noted from derivation above that once we set the initial joint $p(r_0)$, what remains to do is to efficiently update UPM predictive and compute changepoint prior

#### Conjugacy

The computation of UPM predictive leverages `conjugate model`

Assume we have observations $D$, model parameters $\theta$ and hyperparameters $\alpha$. Then the prior predictive distribution marginalized over parameters can be written as

$$p(x|\alpha)=\int p(x|\theta)p(\theta |\alpha)d\theta$$

where $p(x|\theta)$ is `predictive model` given parameters and $(\theta |\alpha)$ is the prior of `parameters`

This is called `prior predictive distribution` because this is the prediction `before` we observe any data (that is, $D$ is not taken into account)

Similarly, we can write `posterior predictive distribution` as

$$
\begin{align*}
p(x|D, \alpha)&=\int p(x|\theta)p(\theta |D, \alpha)d\theta \\
\end{align*}
$$

A wonderful property of `conjugate` model is that the prior distribution and posterior distribution are of the same form. Therefore, if the prior and posterior of parameter are conjugate w.r.t. certain likelihood function, then we have

$$p(\theta|D, \alpha) = p(\theta | \alpha')$$

and

$$
\begin{align*}
p(x|D, \alpha)&=\int p(x|\theta)p(\theta |D, \alpha)d\theta \\
&=\int p(x|\theta)p(\theta |\alpha')d\theta\\
&=p(x|\alpha')
\end{align*}
$$

That is, the posterior predictive distribution is in the `same form` as the prior predictive with only changed hyperparameters $\alpha'$

This allows us to bypass the whole integration thing provided that we can compute $\alpha'$

One family of conjugate model that is particularly attractive to efficiently compute $\alpha'$ is the exponential family

##### Example of Gaussian

As an example, consider a Gaussian distribution with parameter $\mu$ and `fixed` variance $\sigma^2$, where $\mu$ is determined by two hyperparameters $\mu_0$ and $\sigma_0^2$, or with `prior` $\mu \sim N(\mu_0, \sigma_0^2)$

If we have $n$ data points $x_{1:n}$, we can write out the `likelihood`

$$
\begin{align*}L(x_{1:n}|\mu, \sigma^2)&=\prod_{i=1}^n p(x_i|\mu) \\
&=\left(\frac{1}{\sqrt{2\pi\sigma^2}}\right)^n \exp\left[-\frac{1}{2\sigma^2}\sum_{i=1}^n(x_i-\mu)^2\right]
\end{align*}$$

We can now write the `joint`

$$
\begin{align*}
p(x_{1:n}, \mu) &= \frac{1}{\sqrt{2\pi \sigma_0^2}}\exp\left[-\frac{1}{2\sigma_0^2}(\mu-\mu_0)^2\right]\left(\frac{1}{\sqrt{2\pi\sigma^2}}\right)^n \exp\left[-\frac{1}{2\sigma^2}\sum_{i=1}^n(x_i-\mu)^2\right]\\
&\propto \exp\left[-\frac{1}{2\sigma_0^2}\left(\mu^2-2\mu \mu_0+\mu_0^2\right)-\frac{1}{2\sigma^2}\sum_{i=1}^n\left(x_i^2-2x_i\mu+\mu^2\right)\right]\\
&=\exp \left[-\frac{\mu^2}{2\sigma_0^2}+\frac{\mu\mu_0}{\sigma_0^2}-\frac{\mu_0^2}{2\sigma_0^2}-\frac{1}{2\sigma^2}\sum_{i=1}^n x_i^2 +\frac{\mu}{\sigma^2}\sum_{i=1}^nx_i -\frac{\mu^2}{2\sigma^2}n\right] \\
&\propto \exp\left[-\frac{\mu^2\sigma^2+\mu^2\sigma_0^2n}{2\sigma_0^2\sigma^2}+\frac{\mu\mu_0\sigma^2+\mu\sigma_0^2\sum_{i=1}^nx_i}{\sigma_0^2\sigma^2}\right] \\
&=\exp\left[-\frac{1}{2\sigma_0^2\sigma^2}\left((\sigma^2+\sigma_0^2n)\mu^2-2(\mu_0\sigma^2+\sigma_0^2\sum_{i=1}^nx_i)\mu\right)\right]\\
&=\exp\left[-\frac{1}{2\sigma_0^2\sigma^2}\left(a\mu^2+b\mu+c\right)\right] \\
&=\exp\left[-\frac{1}{2\sigma_0^2\sigma^2}\left(a(\mu-d)^2+e\right)\right]\\
a&=\sigma^2+\sigma_0^2n \\
d&=-\frac{b}{2a}=\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma^2+\sigma_0^2n}\\
e&=c-\frac{b^2}{4a}=-\frac{(\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i)^2}{\sigma^2+\sigma_0^2n} \\
&=\exp\left[-\frac{1}{2\sigma_0^2\sigma^2}\left((\sigma^2+\sigma_0^2n)\left(\mu-\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma^2+\sigma_0^2n}\right)^2-\frac{(\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i)^2}{\sigma^2+\sigma_0^2n}\right)\right] \\
&\propto \exp\left[-\frac{\sigma^2+\sigma_0^2n}{2\sigma_0^2\sigma^2}\left(\mu-\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma^2+\sigma_0^2n}\right)^2\right]
\end{align*}
$$

Using Bayes' rule, we know that `posterior` is proportional to this joint and therefore

$$p(\mu|x_{1:n})\propto \exp\left[-\frac{\sigma^2+\sigma_0^2n}{2\sigma_0^2\sigma^2}\left(\mu-\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma^2+\sigma_0^2n}\right)^2\right]$$

which is a Gaussian distribution as well

Therefore, the posterior `variance` is

$$\sigma_n^2 = \left[\frac{\sigma_0^2\sigma^2}{\sigma^2+\sigma_0^2}\right]^2$$

or

$$
\begin{align*}
\frac{1}{\sigma_n^2} &=\frac{\sigma^2+\sigma_0^2n}{\sigma_0^2\sigma^2} \\
&=\frac{\sigma^2}{\sigma_0^2\sigma^2}+\frac{\sigma_0^2}{\sigma^2\sigma_0^2}n \\
&=\frac{1}{\sigma_0^2}+\frac{1}{\sigma^2}n
\end{align*}
$$

and

$$\sigma_n^2 =\frac{\sigma_0^2\sigma^2}{\sigma^2+\sigma_0^2n}= \frac{1}{\frac{1}{\sigma_0^2}+\frac{1}{\sigma^2}n}$$

The posterior `mean` is

$$
\begin{align*}
\mu_n &= \frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma^2+\sigma_0^2n} \\
&=\frac{\sigma^2}{\sigma^2+\sigma_0^2n}\mu_0+\frac{\sigma_0^2}{\sigma^2+\sigma_0^2n}\sum_{i=1}^nx_i \\
&=\sigma_n^2\left(\frac{\mu_0}{\sigma_0^2}+\frac{1}{\sigma^2}\sum_{i=1}^nx_i\right)
\end{align*}$$

Recall that $\frac{1}{n}\sum_{i=1}^nx_i$ is the `maximum likelihood` estimate of $\mu$

So the posterior mean is a weighted sum of prior mean $\mu_0$ and the ML estimate $\mu_{ML}$


##### Derivation from perspective of optimization

Because the posterior $p(\mu| x_{1:n})$ is Gaussian, therefore, the center of posterior density is also the maximum-a-posteriori (MAP) estimate of the mean $\mu_n=\mu_{\text{MAP}}$

If we just want to get this MAP estimate, we can take derivative of $\log p(\mu|x_{1:n})$ or for calculation, the log of joint $\log p(x_{1:n}, \mu)$ (since the difference in constant vanishes after differentiation)

$$
\begin{align*}
\frac{d}{d\mu}\log p(x_{1:n}, \mu)&=\frac{d}{d\mu}\left[-\frac{1}{2\sigma_0^2}(\mu-\mu_0)^2-\frac{1}{2\sigma^2}\sum_{i=1}^n(x_i-\mu)^2+\text{const.}\right] \\
&=\frac{d}{d\mu}\left[-\frac{1}{2\sigma_0^2}(\mu^2-2\mu\mu_0)-\frac{1}{2\sigma^2}\left(n\mu^2-2\mu\sum_{i=1}^nx_i\right)+\text{const.}\right] \\
& \text{group } \mu^2 \text{ and } \mu \\
&=\frac{d}{d\mu}\left[-\frac{\sigma^2+n\sigma_0^2}{2\sigma_0^2\sigma^2}\mu^2+\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma_0^2\sigma^2}\mu +\text{const.}\right]
\end{align*}
$$

and set to zero, we have

$$\frac{\sigma^2+n\sigma_0^2}{\sigma_0^2\sigma^2}\mu=\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma_0^2\sigma^2} $$

Solve for the MAP estimate

$$\mu_{\text{MAP}}=\frac{\sigma^2\mu_0+\sigma_0^2\sum_{i=1}^nx_i}{\sigma^2+n\sigma_0^2}$$

which is the same as the posterior mean $\mu_n$ we get earlier

Both approaches, taking the derivative or completing the square, are mathematically identical

The latter just skips writing the derivative because the result can be read off immediately once the quadratic is centered

Same can be done for posterior variance

Since the posterior is Gaussian, we know

$$\log p(\mu|x_{1:n})=-\frac{1}{2\sigma_n^2}(\mu-\mu_n)^2+\text{const.}$$

Take second derivative w.r.t. $\mu$

$$
\begin{align*}
\left.\frac{d^2}{d\mu^2}\log p(\mu|x_{1:n})\right|_{\mu=\mu_n}
&=\left.\frac{d^2}{d\mu^2}\log p(\mu,x_{1:n})\right|_{\mu=\mu_n}\\
&=-\frac{1}{\sigma_n^2} \\
&=-\frac{\sigma^2+n\sigma_0^2}{\sigma_0^2\sigma^2}
\end{align*}
$$

and we get posterior variance as before

$$\sigma_n^2 =\frac{\sigma_0^2\sigma^2}{\sigma^2+n\sigma_0^2}$$

#### Exponential family

For exponential family, standard distributions with conjugate priors can be cast into a canonical form (we can think of it as `likelihood`)

$$p(x|\eta)=h(x)g(\eta)\exp \left[\eta^T u(x)\right]$$

where
  - $h(x)$ is `underlying measure` carrying every factor that does not involve $\eta$
  - $u(x)$ is sufficient-statistic of `data`
  - $\eta$ is `natural-parameter`
  - $g(\eta)$ is normalizer

$$\begin{align*}
g(\eta) \int h(x) \exp\left[\eta^T u(x)\right]dx=1
\end{align*}$$

##### Sufficient statistic

For data $x_{1:n}$, the idea of sufficient statistic is that we can use $\sum u(x_{i:n)$ to compute maximum likelihood eastimates of natural parameters

Use canonical form, we can write the log-likelihood as

$$
\begin{align*}
\log p(x_{1:n}|\eta)&=\log \prod_{i=1}^n p(x_i|\eta)\\
&=\log \left[\left(\prod_{i=1}^nh(x_i)\right)g(\eta)^n\exp\left(\eta^T\sum_{i=1}^nu(x_i)\right)\right]\\
&=\log \left(\prod_{i=1}^nh(x_i)\right)+n\log g(\eta)+\eta^T\sum_{i=1}^nu(x_i)
\end{align*}
$$

Take gradient w.r.t. $\eta$ and set to zero

$$\nabla n\log g(\eta_{\text{ML}})+\sum_{i=1}^nu(x_i)=0$$

or

$$-\nabla \log g(\eta_{\text{ML}}) = \frac{1}{n}\sum_{i=1}^nu(x_i)$$

Note that since sufficient statistic is a running sum, we can compute it `incrementally` as new data arrives

##### Moments through differentiation

Denote log normalizer

$$A(\eta)=-\log g(\eta)=\log \int h(x)\exp \left[\eta^Tu(x)\right]dx$$

Take gradient w.r.t. $\eta$

$$\nabla A(\eta)=\frac{1}{Z(\eta)}\nabla Z(\eta)$$

where

$$Z(\eta)=\exp[A(\eta)]=\int h(x) \exp\left[\eta^Tu(x)\right]dx$$

Under some mild condition, we can write

$$
\begin{align*}
\nabla Z(\eta)&=\int h(x)u(x)\exp\left[\eta^Tu(x)\right]dx \\
&=\int u(x)Z(\eta)\frac{h(x)\exp\left[\eta^Tu(x)\right]}{Z(\eta)}dx\\
& \text{the fraction is a density on }x\\
&=\mathbb{E}\left[Z(\eta)u(x)\right]\\
&=Z(\eta)\mathbb{E}\left[u(x)\right]
\end{align*}$$

Therefore

$$\nabla A(\eta)=\frac{1}{Z(\eta)}\nabla Z(\eta)=\mathbb{E}\left[u(x)\right]$$

##### Example of Gaussian

Again, using Gaussian as example, with both mean and variance as parameters

$$
\begin{align*}
p(x|\mu, \sigma^2)&=\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-\frac{1}{2\sigma^2}(x-\mu)^2\right] \\
&=\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-\frac{1}{2\sigma^2}x^2+\frac{\mu}{\sigma^2}x-\frac{\mu^2}{2\sigma^2}\right]\\
&=\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[-\frac{\mu^2}{2\sigma^2}\right]\exp\left[\frac{\mu}{\sigma^2}-\frac{1}{2\sigma^2}x^2\right]
\end{align*}
$$

By split like this, we can introduce sufficient statistic and natural parameter

$$u(x)=\begin{bmatrix}x \\ x^2\end{bmatrix}, \eta(\mu, \sigma^2)=\begin{bmatrix}\eta_1\\ \eta_2\end{bmatrix}=\begin{bmatrix}\frac{\mu}{\sigma^2}\\ -\frac{1}{2\sigma^2}\end{bmatrix}$$

In addition

$$h(x)=\frac{1}{\sqrt{2\pi}}, g(\eta)=\frac{1}{\sigma}\exp\left[-\frac{\mu^2}{2\sigma^2}\right]$$

Notice that

$$\frac{1}{\sigma}=\sqrt{-2\eta_2}$$

and

$$-\frac{\mu^2}{2\sigma^2}=\frac{\eta_1^2}{4\eta_2}$$

we can rewrite the normalizer as

$$g(\eta)=\sqrt{-2\eta_2}\frac{\eta_1^2}{4\eta_2}$$